In [1]:
from math import *
import random

In [2]:
landmarks=[[20.0,20.0],[80.0,80.0],[20.0,80.0],[80.0,20.0]]
world_size=100.0

In [8]:
class robot:
    def __init__(self):
        self.x=random.Random()*world_size
        self.y=random.Random()*world_size
        self.orientation=random.Random()*2.0*pi
        slef.forward_noise=0.0
        self.turn_noise=0.0
        self.sense_noise=0.0
        
    def set(self,new_x,new_y,new_orientation):
        if (new_x<0 or new_x>= world_size):
            raise ValueError ('X coordinate out of bound' )
        if (new_y<0 or new_y>= world_size):
            raise ValueError ('Y coordinate out of bound')
        if (new_orientation<0 or new_orientation >=(2*pi)):
            raise ValueError ('Orientation must be in [0..2pi]')
        
        self.x=float(new_x)
        self.y=float(new_y)
        self.orientation=float(new_orientation)
     
    def set_noise(self,nnew_f_noise,new_t_noise,new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
        
    def sense(self):
        Z=[]
        for i in range(len(landmarks)):
            dist=sqrt((self.x-landmarks[i][0])**2+(self.y-landmarks[i][1])**2)
            dist +=random.gauss(0.0,self.sense_noise)
            Z.append(dist)
        return Z
    def move(self,turn,forward):
        if (forward<0):
            raise ValueError('Robot cant move backwards')   
        
        # turn, and add randomness to the turning command
        orientation=self.orientation+float(turn)+random.gauss(0.0,self.turn_noise)
        orientation %=2*pi
        
        # move, and add randomness to the motion command
        dist=float(forward)+random.gauss(self.forward_noise)
        x=self.x+(cos(orientation)*dist)
        y=self.y+(sin(orientation)*dist)
        x=x%world_size  # cyclic truncate
        y=y%world_size

        # set particle
        res=robot()
        res.set(x,y,orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    def Gaussian(self,mu,sigma,x):
       # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(-((mu-x)**2)/sigma**2/2.0)/sqrt(2.0*pi*(sigma**2))

    def measurement(self,measurement):
        
        # calculates how likely a measurement should be
        prob=1.0
        for in range(len(measurement)):
            dist=sqrt((self.x-measurement[i][0])**2+(self.y-measurement[i][1]))
            prob= *=slf,